In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

2023-04-20 19:06:43.220285: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[]


In [3]:
# Create mappings
mapping = MusicMapping.create()

# Load model
model = MukkeBudeTransformer.load(mapping=mapping, name="Bach_soloMelodie_transformer")

In [4]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep", max_length=300, probability=0.8)
new_song_str = generated_song.numpy().decode("utf-8")
print(f"CLEAN SONG: {new_song_str}\n\n")

print(new_song_str[:50])

prompt_tokens: [ 2 23 17 20 17 39 17 55 17  4 17 23 17 20 17 39 17 30 17  4 17 22 17 20
 17 25 17 38 17  4]


RecursionError: Exception encountered when calling layer "word_piece_tokenizer" "                 f"(type WordPieceTokenizer).

maximum recursion depth exceeded while calling a Python object

Call arguments received by layer "word_piece_tokenizer" "                 f"(type WordPieceTokenizer):
  • args=('tf.Tensor(shape=(300,), dtype=int32)',)
  • mode='detokenize'
  • training=False
  • kwargs=<class 'inspect._empty'>

In [ ]:
# Convert to music21
new_song_ints = mapping.numericalize(new_song_str.split(" "))
new_song_ints = np.array(new_song_ints)

print(new_song_ints[:50])
print(mapping.textify(new_song_ints[:50]))

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=100)
path = Path("generated_song_bach.mid")
utils.write_midi(new_song, path)
utils.write_musicxml(new_song, "generated_song_bach.musicxml")